## В этом домашнем задании вы сделаете первые шаги в мире линейной бинарной классификации!

In [1]:
import pandas as pd
import numpy as np
import warnings

#### Задание 1

Мы будем работать с данными **Microsoft Malware Detection**

Таргетом будет последний столбец `HasDetection`, который принимает значения $\{0,\, 1\}$ в случае отсутствия или наличия вируса на компьютере соответственно. Признаками будут выступать всевозможные характеристики компьютера.

In [2]:
data = pd.read_csv('E:/train_hw_10.csv')

C:\Users\MobileMarket\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (28) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
data.head()

,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,...,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
0,0000028988387b115f69f31a3bf04f09,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1735.0,0,7.0,0,NaN,53447.0,...,36144.0,0,NaN,0.0,0,0,0.0,0.0,10.0,0
1,000007535c3f730efa9ea0b7ef1bd645,win8defender,1.1.14600.4,4.13.17134.1,1.263.48.0,0,7.0,0,NaN,53447.0,...,57858.0,0,NaN,0.0,0,0,0.0,0.0,8.0,0
2,000007905a28d863f6d0d597892cd692,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1341.0,0,7.0,0,NaN,53447.0,...,52682.0,0,NaN,0.0,0,0,0.0,0.0,3.0,0
3,00000b11598a75ea8ba1beea8459149f,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1527.0,0,7.0,0,NaN,53447.0,...,20050.0,0,NaN,0.0,0,0,0.0,0.0,3.0,1
4,000014a5f00daa18e76b81417eeb99fc,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1379.0,0,7.0,0,NaN,53447.0,...,19844.0,0,0.0,0.0,0,0,0.0,0.0,1.0,1


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 83 columns):
 #   Column                                             Non-Null Count   Dtype  
---  ------                                             --------------   -----  
 0   MachineIdentifier                                  200000 non-null  object 
 1   ProductName                                        200000 non-null  object 
 2   EngineVersion                                      200000 non-null  object 
 3   AppVersion                                         200000 non-null  object 
 4   AvSigVersion                                       200000 non-null  object 
 5   IsBeta                                             200000 non-null  int64  
 6   RtpStateBitfield                                   199236 non-null  float64
 7   IsSxsPassiveMode                                   200000 non-null  int64  
 8   DefaultBrowsersIdentifier                          9703 non-null    float6

Удалите константные признаки и признаки `ProductName` `MachineIdentifier`

In [5]:
### Your code is here

data = data.drop(['ProductName', 'MachineIdentifier'], axis = 1)

In [6]:
const = []
for col in data.columns:
    if len(data[col].unique()) == 1:
        const.append(col)


In [7]:
data = data.drop(const, axis = 1)

Посмотрите на соотношение классов в таргете. Все ли хорошо?

In [8]:
print(sum(data['HasDetections'] == 1), '- positive class,')
print(sum(data['HasDetections'] == 0), '- negative class')

100060 - positive class,
99940 - negative class


Ответьте на вопрос: почему с вашей точки зрения важно иметь представление о балансе классов в ваших данных?

Избавьтесь от пропусков в данных! 

Новый для нас прием: признаки с более чем половиной пропусков следует удалить.

Согласитесь, если в вашей колонке среди 100 объектов всего лишь у 2 есть какое-то непропущенное значение, странно все остальные заполнять средним от этих двух чисел. Такие "редкие" признаки лучше вообще опустить!


В категориальных колонках заменим отсутствующую категорию просто некоторой новой и назовем ее `NaN`.

А в числовых, ради разнообразия, заполним пропуски медианным значением.

In [9]:
### Your code is here
#удаляем те колонки, где пропущено больше половины занчений 
no_need = []
for col in data.columns:
    if data[col].isna().sum() >= data[col].count():
        no_need.append(col)
no_need

['DefaultBrowsersIdentifier',
 'PuaMode',
 'Census_ProcessorClass',
 'Census_InternalBatteryType',
 'Census_IsFlightingInternal',
 'Census_ThresholdOptIn',
 'Census_IsWIMBootEnabled']

In [10]:
data = data.drop(no_need, axis = 1)

In [11]:
### Посмотрим на некатегориальные колонки

numeric_columns = data.loc[:,data.dtypes!=np.object].columns

for col in numeric_columns:
    data[col] = data[col].fillna(data[col].mean())
   

C:\Users\MOBILE~1\AppData\Local\Temp/ipykernel_12000/503884598.py:3: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  numeric_columns = data.loc[:,data.dtypes!=np.object].columns


In [12]:
### Посмотрим на категориальные колонки

categorical_columns = data.loc[:,data.dtypes==np.object].columns

for col in categorical_columns:
    data[col] = data[col].fillna('NaN')


C:\Users\MOBILE~1\AppData\Local\Temp/ipykernel_12000/3356536803.py:3: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  categorical_columns = data.loc[:,data.dtypes==np.object].columns


Создайте копию полученного датафрейма и положите ее в переменную data_2. Понадобится в следующих заданиях.

In [13]:
data_2 = data.copy()

Так же поработаем над всеми категориальными колонкам перед запуском непосредственно моделей.

Провернем самый базовый и наглый метод - несмотря на количество уникальных значений в каждой категории, просто применим ко всей категориальной части датасета `OneHotEncoding`

In [14]:
### Your code is here
### Делаем OneHotEncoding и конкатим с processed_data

""" 
for col in categorical_columns:
    onehot = pd.get_dummies(data_2[col], drop_first=True)
    onehot.columns = [f"{col}_{x}" for x in onehot.columns]
    
    data_2 = pd.concat((data_2, onehot), axis=1)
    data_2 = data_2.drop(col, axis=1)
    
"""

' \nfor col in categorical_columns:\n    onehot = pd.get_dummies(data_2[col], drop_first=True)\n    onehot.columns = [f"{col}_{x}" for x in onehot.columns]\n    \n    data_2 = pd.concat((data_2, onehot), axis=1)\n    data_2 = data_2.drop(col, axis=1)\n    \n'

In [15]:
#data_2.head() #должно быть 6114

###  Разделим выборку на тренировочную и тестовую

P.S. в задачах классификации, как и в задаче регрессии, можно использовать технологию Кросс-Валидации. Например, по одному из двух следующих сценариев:

1) Отделить валидацию и тест, произвести подбор лучшей модели с помощью `K-Fold` на валидации, финально обучить выбранную модель на всей валидации и замерить качество на заранее отложенном финальном тесте!

2) Всю выборку назвать валидационной и на ней применить `K-Fold` без финального замера.

В этой домашней работе попросим Вас быть еще проще! :)
Реализуем просто технологию отложенной выборки в пропорции 3:1

In [16]:
"""

from sklearn.model_selection import train_test_split 

x = data_2.drop(columns=['HasDetections'])
y = data_2['HasDetections']

x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                    test_size=0.25,
                                                    shuffle=True,
                                                    random_state=1)

"""

"\n\nfrom sklearn.model_selection import train_test_split \n\nx = data_2.drop(columns=['HasDetections'])\ny = data_2['HasDetections']\n\nx_train, x_test, y_train, y_test = train_test_split(x, y,\n                                                    test_size=0.25,\n                                                    shuffle=True,\n                                                    random_state=1)\n\n"

Соберите `Pipeline`, реализовав в нем 2 шага: стандартизация данных через `StandardScaler` и обучение логистической регрессии с помощью `LogisticRegression`, положите результаты в переменную `pipe`, а в классе модели `LogisticRegression` укажите параметр `penalty='none'`

In [17]:
"""
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

### Your code is here

pipe = Pipeline([('scaler', StandardScaler()), ('LR', LogisticRegression(penalty='none'))]) 
#pipe.fit(x_train, y_train)
"""

"\nfrom sklearn.preprocessing import StandardScaler\nfrom sklearn.linear_model import LogisticRegression\nfrom sklearn.pipeline import Pipeline\n\n### Your code is here\n\npipe = Pipeline([('scaler', StandardScaler()), ('LR', LogisticRegression(penalty='none'))]) \n#pipe.fit(x_train, y_train)\n"

Чтобы замерить качество работы такой модели на трейне и на тесте воспользуемся функцией `cross_validate`

Вопрос: что передавать ей в параметр cv? Ведь мы уже разделили нашу выборку на трейн и тест (имеем всего 1 fold). Для этого можно просто передать список от кортежа, содержащего индексы тренировочных и тестовых объектов.

In [18]:
"""
from sklearn.model_selection import cross_validate
import datetime

custom_cv = [(x_train.index.to_list(), x_test.index.to_list())]

begin_time = datetime.datetime.now()

cv_result_pipe = cross_validate(pipe, x, y, scoring='accuracy',
                                cv=custom_cv, return_train_score=True)


print(f"Accuracy на трейне: {np.mean(cv_result_pipe['train_score']).round(3)}")
print(f"Accuracy на тесте: {np.mean(cv_result_pipe['test_score']).round(3)}")

print(f"Время работы алгоритма: {datetime.datetime.now() - begin_time}")
"""

'\nfrom sklearn.model_selection import cross_validate\nimport datetime\n\ncustom_cv = [(x_train.index.to_list(), x_test.index.to_list())]\n\nbegin_time = datetime.datetime.now()\n\ncv_result_pipe = cross_validate(pipe, x, y, scoring=\'accuracy\',\n                                cv=custom_cv, return_train_score=True)\n\n\nprint(f"Accuracy на трейне: {np.mean(cv_result_pipe[\'train_score\']).round(3)}")\nprint(f"Accuracy на тесте: {np.mean(cv_result_pipe[\'test_score\']).round(3)}")\n\nprint(f"Время работы алгоритма: {datetime.datetime.now() - begin_time}")\n'

Что можете сказать про время работы алгоритма?

Очевидно, оно достаточно большие. Уж тем более для линейных моделей.

Такое как раз-таки происходит из-за того, что количество фичей, который мы передали нашей модели - гигантское! Классу требуется много времени и памяти, чтобы обработать датасет.

Поэтому те колонки, в которых количество уникальных категорий превышает какое-то адекватное число, следует кодировать иначе, нежели с помощью технологии `One-Hot-Encoding`.

Теперь вы верите, что более умные кодировки зачастую прям необходимы! Раньше мы этот факт не демонстрировали!

Дело еще вот в чем: в классе `LogisticRegression`, как и, например, `Lasso`, есть параметр, ограничивающий максимальное количество итераций во время обучения модели. Так, если данных много и итераций тоже ожидается большое число, найденная разделяющая гиперплоскость может оказаться не самой лучшей, так как наш алгоритм (будь то градиентный спуск или любой иной) просто 'не доползет'. 

#### Задание 2

Теперь попробуем другой метод кодирования категориальных колонок, а именно счётчики.
Построем ту же модель и на том же разделении, просто заново иначе переобработаем датасет. 

Для тех категориальных признаков, у которых количество уникальных значений в колоночках больше 5, применим `MeanTargetEncoding`.

Для остальных оставим любимый `OneHotEncoding` (как делали на практике и в предыдущем уроке).

In [19]:
### Your code is here
### Произведем mean-target-encoding колонок с количсетвом уникальных значений больше 5
categorical_columns = data.loc[:,data.dtypes==np.object].columns
for col in categorical_columns:
    if len(np.unique(data[col])) > 5:
        data[col] = data[col].map(data.groupby([col])['HasDetections'].mean())

C:\Users\MOBILE~1\AppData\Local\Temp/ipykernel_12000/3136479468.py:3: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  categorical_columns = data.loc[:,data.dtypes==np.object].columns


In [20]:
#для оставшихся колонок One-hot encoding
### Делаем OneHotEncoding и конкатим с processed_data

for col in categorical_columns:
    if int(len(np.unique(data[col]))) <= int(5):
        onehot = pd.get_dummies(data[col], drop_first=True)
        onehot.columns = [f"{col}_{x}" for x in onehot.columns]
    
        data = pd.concat((data, onehot), axis=1)
        data = data.drop(col, axis=1)

Опять обучим модель, пока что без изменений! Скажите, стало ли быстрее? А что с качеством?

In [21]:
### Your code is here
from sklearn.model_selection import train_test_split 

X_2 = data.drop(columns=['HasDetections'])
Y_2 = data['HasDetections']

x_train, x_test, y_train, y_test = train_test_split(X_2, Y_2,
                                                    test_size=0.25,
                                                    shuffle=True,
                                                    random_state=1)
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_validate
import datetime

### Your code is here

pipe = Pipeline([('scaler', StandardScaler()), ('LR', LogisticRegression(penalty='none'))]) 
#pipe.fit(x_train, y_train)


custom_cv = [(x_train.index.to_list(), x_test.index.to_list())]

begin_time = datetime.datetime.now()

cv_result_pipe = cross_validate(pipe, X_2, Y_2, scoring='accuracy',
                                cv=custom_cv, return_train_score=True)


print(f"Accuracy на трейне: {np.mean(cv_result_pipe['train_score']).round(3)}")
print(f"Accuracy на тесте: {np.mean(cv_result_pipe['test_score']).round(3)}")

print(f"Время работы алгоритма: {datetime.datetime.now() - begin_time}")


Accuracy на трейне: 0.638
Accuracy на тесте: 0.638
Время работы алгоритма: 0:00:06.520003


In [ ]:
select_dtypes(exclude=['float64', 'int64'])

#### Задание 3: Регуляризация

Как и в моделях регрессии, решая задачу классификации, можем штрафовать минимизируемый функционал за большие веса, добавив к нему L1 или L2 норму весов (все как раньше!).

Для этого в изначальном классе `LogisticRegression` изменить параметр `penalty` на l1 или l2 соответственно. Выберите второй вариант! Можно воспользоваться методом `set_params` и применить его к `pipe`.

In [23]:
### Your code is here
pipe = Pipeline([('scaler', StandardScaler()), ('LR', LogisticRegression(penalty='l2'))]) 
#pipe.set_params(penalty = 'l2')
custom_cv = [(x_train.index.to_list(), x_test.index.to_list())]

begin_time = datetime.datetime.now()

cv_result_pipe = cross_validate(pipe, X_2, Y_2, scoring='accuracy',
                                cv=custom_cv, return_train_score=True)


print(f"Accuracy на трейне: {np.mean(cv_result_pipe['train_score']).round(3)}")
print(f"Accuracy на тесте: {np.mean(cv_result_pipe['test_score']).round(3)}")

print(f"Время работы алгоритма: {datetime.datetime.now() - begin_time}")

Accuracy на трейне: 0.638
Accuracy на тесте: 0.638
Время работы алгоритма: 0:00:06.002004


Теперь наша модель будет строить логистическую регрессию с L2 регуляризатором! Помним, что у регуляризируемых моделей есть гиперпараметр, контролирующий силу регуляризации, который выбирается ДО запуска метода fit, то есть заранее, когда модель еще не обучена. 

Конечно же, выбор этого параметра влияет итоговые результаты. Хочется поперебирать различные параметры регуляризации и найти такой, при котором качество на тесте окажется лучшим! 

Сгенерируем массив гиперпараметра, которые планируем перебрать:

In [24]:
alphas = np.linspace(0.01, 100, 100)
alphas

array([1.000e-02, 1.020e+00, 2.030e+00, 3.040e+00, 4.050e+00, 5.060e+00,
       6.070e+00, 7.080e+00, 8.090e+00, 9.100e+00, 1.011e+01, 1.112e+01,
       1.213e+01, 1.314e+01, 1.415e+01, 1.516e+01, 1.617e+01, 1.718e+01,
       1.819e+01, 1.920e+01, 2.021e+01, 2.122e+01, 2.223e+01, 2.324e+01,
       2.425e+01, 2.526e+01, 2.627e+01, 2.728e+01, 2.829e+01, 2.930e+01,
       3.031e+01, 3.132e+01, 3.233e+01, 3.334e+01, 3.435e+01, 3.536e+01,
       3.637e+01, 3.738e+01, 3.839e+01, 3.940e+01, 4.041e+01, 4.142e+01,
       4.243e+01, 4.344e+01, 4.445e+01, 4.546e+01, 4.647e+01, 4.748e+01,
       4.849e+01, 4.950e+01, 5.051e+01, 5.152e+01, 5.253e+01, 5.354e+01,
       5.455e+01, 5.556e+01, 5.657e+01, 5.758e+01, 5.859e+01, 5.960e+01,
       6.061e+01, 6.162e+01, 6.263e+01, 6.364e+01, 6.465e+01, 6.566e+01,
       6.667e+01, 6.768e+01, 6.869e+01, 6.970e+01, 7.071e+01, 7.172e+01,
       7.273e+01, 7.374e+01, 7.475e+01, 7.576e+01, 7.677e+01, 7.778e+01,
       7.879e+01, 7.980e+01, 8.081e+01, 8.182e+01, 

Чтобы отобрать среди данного массива гиперпараметров лучший, воспользуйтесь конструкцией `GridSearchCV` из `sklearn`

In [26]:
from sklearn.model_selection import GridSearchCV

param_grid ={
    'LR__C': alphas
}


### Передадим в GridSearchCV

search = GridSearchCV(pipe, param_grid, 
                      cv=custom_cv, scoring='neg_mean_squared_error')

search.fit(X_2, Y_2)

print(f"Best parameter (CV score={search.best_score_:.5f}):")
print(search.best_params_)


KeyboardInterrupt: 

In [ ]:
pipe.set_params(Lasso__alpha=search.best_params_['Lasso__alpha'])

#### Задание 4: Бонус

Как вы знаете, подбор признаков является одной из самых важных частей машинного обучения. Сейчас вы попробуете на основе имеющихся признаков сгенерировать новые. В качестве новых признаков будем использовать мономы 2 степени. Можно использовать регуляризацию различного рода, выбор энкодера тоже за вами. Ваша задача - добиться качества `0.65` на тестовой выборке

In [19]:
### Your code is here



In [15]:
a = []
def sort(tup):
# место для вашего кода
 
    for i in tup:
        a.append(i)
        
    

sort((-1, 0, 4, 543, 80, -90))
print(sorted(a))

[-90, -1, 0, 4, 80, 543]


In [37]:
tup = tuple(map(int, input().split()))
a = []
def fu(tup):
    tup_sum = 0
    for i in tup:
        a.append(i)
  
    
fu(tup)

9 72 64 6 44 76 45 12 22 10


In [38]:
print(sum(a))

360
